# Putting it all together

So far we have done the following on the prior Notebooks:

- **Notebook 01**: "cogsrch-index-files" 인덱스에 강화된 PDF가 포함된 Azure 검색 엔진을 로드했습니다.
- **Notebook 02**: LLM의 유틸리티 체인을 사용하여 답변 생성을 향상시키기 위해 AzureOpenAI GPT 모델을 추가했습니다.
- **Notebook 03**: 대용량/복잡한 PDF 정보가 포함된 인덱스("cogsrch-index-books")를 수동으로 로드했습니다.
- **Notebook 04**: 대화형 Chat Bot을 강화하기 위해 시스템에 메모리를 추가했습니다.
- **Notebook 05**: 에이전트와 Tool(도구)를 도입하고 검색 엔진을 통해 RAG를 수행할 수 있는 최초의 Skill/Agent를 구축했습니다.

한 가지 더 놓친 것이 있습니다: **이 모든 기능을 어떻게 하면 매우 스마트한 GPT 스마트 검색 엔진 채팅 봇으로 통합할 수 있을까요?**

우리는 질문을 받고, 어떤 Tool을 사용할지 생각하고, 답을 얻을 수 있는 가상 비서를 원합니다. 목표는 정보의 출처(검색 엔진, Bing 검색, SQL 데이터베이스, CSV 파일, JSON 파일, API 등)에 관계없이 어시스턴트가 올바른 Tool을 사용하여 질문에 올바르게 답변할 수 있도록 하는 것입니다.<br>.

에이전트는 사용할 수 있는 더 많은 Tool을 구축할 수 있습니다.

이 노트북에서는 '두뇌' 에이전트(마스터 에이전트라고도 함)를 만들 것입니다.

1) 질문을 이해하고 사용자와 상호 작용합니다. 
2) 다른 소스에 연결된 다른 전문 에이전트와 대화합니다.
3) 답변을 얻으면 사용자에게 전달하거나 전문 에이전트가 직접 전달하도록 합니다.

This is the same concept of [AutoGen](https://www.microsoft.com/en-us/research/blog/autogen-enabling-next-generation-large-language-model-applications/): Agents talking to each other.

![image](./images/AutoGen_Fig1.png)

In [ ]:
import os
import random
import json
import requests
from operator import itemgetter
from typing import Union, List
from langchain_openai import AzureChatOpenAI
from langchain.agents import AgentExecutor, Tool, create_openai_tools_agent
from langchain_community.chat_message_histories import ChatMessageHistory, CosmosDBChatMessageHistory
from langchain.callbacks.manager import CallbackManager
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import ConfigurableFieldSpec, ConfigurableField
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain.output_parsers import JsonOutputToolsParser
from langchain_core.runnables import (
    Runnable,
    RunnableLambda,
    RunnableMap,
    RunnablePassthrough,
)

#custom libraries that we will use later in the app
from common.utils import (
    DocSearchAgent, 
    CSVTabularAgent, 
    SQLSearchAgent, 
    ChatGPTTool, 
    BingSearchAgent, 
    APISearchAgent, 
    reduce_openapi_spec
)
from common.callbacks import StdOutCallbackHandler
from common.prompts import CUSTOM_CHATBOT_PROMPT 

from dotenv import load_dotenv
load_dotenv("credentials.env")

from IPython.display import Markdown, HTML, display 

def printmd(string):
    display(Markdown(string))


In [ ]:
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"]

### Get the Tool - DocSearch Agent, ChatGPT (more agents can be included - CSV Agent, SQL Agent, Web Search Agent, ChatGPT, API Agent)

**Consider the following concept:** 기본적으로 특정 작업을 수행하도록 설계된 소프트웨어 개체인 에이전트에는 도구가 탑재될 수 있습니다. 이러한 도구 자체는 각각 고유한 도구 세트를 보유한 다른 에이전트가 될 수 있습니다. 이렇게 하면 도구가 코드 시퀀스에서 사람의 행동에 이르기까지 다양한 계층 구조를 형성하여 상호 연결된 체인을 형성할 수 있습니다. 궁극적으로 특정 작업을 해결하기 위해 협업하는 에이전트와 각각의 도구로 구성된 네트워크를 구축하는 것입니다(이것이 바로 ChatGPT입니다). 이 네트워크는 각 에이전트와 도구의 고유한 기능을 활용하여 작동하며, 작업 해결을 위한 역동적이고 효율적인 시스템을 만듭니다. 

 `common/utils.py` 파일에서 이전 노트북에서 개발했던 각 기능에 대한 에이전트 도구 클래스를 만들었습니다. 

In [ ]:
cb_handler = StdOutCallbackHandler()
cb_manager = CallbackManager(handlers=[cb_handler])

COMPLETION_TOKENS = 2000

# We can run the everything with GPT3.5, but try also GPT4 and see the difference in the quality of responses
# You will notice that GPT3.5 is not as reliable.

llm = AzureChatOpenAI(deployment_name=os.environ["GPT35_DEPLOYMENT_NAME"], 
                      temperature=0, max_tokens=COMPLETION_TOKENS)

# Uncomment below if you want to see the answers streaming
# llm = AzureChatOpenAI(deployment_name=os.environ["GPT35_DEPLOYMENT_NAME"], temperature=0.5, max_tokens=COMPLETION_TOKENS, streaming=True, callback_manager=cb_manager)


In [ ]:
doc_indexes = ["cogsrch-index-files"]
doc_search = DocSearchAgent(llm=llm, indexes=doc_indexes,
                           k=6, reranker_th=1,
                           sas_token=os.environ['BLOB_SAS_TOKEN'],
                           name="docsearch",
                           description="useful when the questions includes the term: docsearch",
                           callback_manager=cb_manager, verbose=False)

In [ ]:
book_indexes = ["cogsrch-index-books"]
book_search = DocSearchAgent(llm=llm, indexes=book_indexes,
                           k=5, reranker_th=1,
                           sas_token=os.environ['BLOB_SAS_TOKEN'],
                           name="booksearch",
                           description="useful when the questions includes the term: booksearch",
                           callback_manager=cb_manager, verbose=False)

In [ ]:
## ChatGPTTool is a custom Tool class created to talk to ChatGPT knowledge
chatgpt_search = ChatGPTTool(llm=llm, callback_manager=cb_manager,
                             name="chatgpt",
                            description="use for general questions, profile, greeting-like questions and when the questions includes the term: chatgpt",
                            verbose=False)

### Variables/knobs to use for customization

지금까지 살펴본 것처럼 GPT 스마트 검색 엔진 애플리케이션의 동작을 변경하기 위해 다이얼을 올리거나 내릴 수 있는 많은 knob가 있으며, 이를 조정할 수 있는 변수가 있습니다.

- <u>llm</u>:
  - **deployment_name**: this is the deployment name of your Azure OpenAI model. This of course dictates the level of reasoning and the amount of tokens available for the conversation. For a production system you will need gpt-4-32k. This is the model that will give you enough reasoning power to work with agents, and enough tokens to work with detailed answers and conversation memory.
  - **temperature**: How creative you want your responses to be
  - **max_tokens**: How long you want your responses to be. It is recommended a minimum of 500
- <u>Tools</u>: To each tool you can add the following parameters to modify the defaults (set in utils.py), these are very important since they are part of the system prompt and determines what tool to use and when.
  - **name**: the name of the tool
  - **description**: when the brain agent should use this tool
- <u>DocSearchAgent</u>: 
  - **k**: The top k results per index from the text search action
  - **similarity_k**: top k results combined from the vector search action
  - **reranker_th**: threshold of the semantic search reranker. Picks results that are above the threshold. Max possible score=4
  
in `utils.py` you can also tune:
- <u>model_tokens_limit</u>: In this function you can edit what is the maximum allows of tokens reserve for the content. Remember that the remaining will be for the system prompt plus the answer

### Test the Tools

In [ ]:
# Test the Document Search Tool with a question that we know it has the answer for
printmd(doc_search.run("what is the responsibility of Manager of Human Resources?"))

In [ ]:
# Test the other index created manually
printmd(book_search.run("Can I move, backup, and restore indexes?"))

In [ ]:
# Test the ChatGPTWrapper Search Tool
printmd(chatgpt_search.run("what is the function in python that allows me to get a random number?"))

### Define what tools are we going to give to our brain agent

Go to `common/utils.py` to check the tools definition and the instructions on what tool to use when

In [ ]:
tools = [doc_search, book_search, chatgpt_search]

# Option 1: Using OpenAI functions as router

질문을 올바른 도구로 라우팅하는 방법이 필요하며, 이를 위한 한 가지 방법은 도구 API(모델 1106 이상)를 통해 OpenAI 모델 함수를 사용하는 것입니다. 이렇게 하려면 이러한 도구/함수를 모델에 바인딩하고 모델이 적합한 도구로 응답하도록 해야 합니다.

이 옵션의 장점은 전문가(Agent Tool)와 사용자 사이에 다른 Agent가 중간에 끼어 있지 않다는 것입니다. 각 에이전트 도구가 직접 응답합니다. 또한 여러 도구를 병렬로 호출할 수 있다는 장점도 있습니다.

**Note**: 이 방법에서는 각 Agent가 동일한 시스템 프로필 프롬프트를 사용하여 동일한 응답 가이드라인을 준수하는 것이 중요합니다.

In [ ]:
llm_with_tools = llm.bind_tools(tools)
tool_map = {tool.name: tool for tool in tools}

In [ ]:
def call_tool(tool_invocation: dict) -> Union[str, Runnable]:
    """Function for dynamically constructing the end of the chain based on the model-selected tool."""
    tool = tool_map[tool_invocation["type"]]
    return RunnablePassthrough.assign(output=itemgetter("args") | tool)

def print_response(result: List):
    for answer in result:
        printmd("**"+answer["type"] + "**" + ": " + answer["output"])
        printmd("----")
      
# .map() allows us to apply a function to a list of inputs.
call_tool_list = RunnableLambda(call_tool).map()
agent = llm_with_tools | JsonOutputToolsParser() | call_tool_list

In [ ]:
result = agent.invoke("hi, how are you, what is your name?")
print_response(result)

In [ ]:
result = agent.invoke("Who is the current president of France?")
print_response(result)

In [ ]:
result = agent.invoke("docsearch,chatgpt, what is the responsibility of Manager of Human Resources?")
print_response(result)

# Option 2: Using a user facing agent that calls the agent tools experts

이 방법을 사용하면 사용자와 대화하고 전문가(에이전트 도구)와도 대화하는 User Facing Agent를 만들 수 있습니다.

### Initialize the brain agent

In [ ]:
agent = create_openai_tools_agent(llm, tools, CUSTOM_CHATBOT_PROMPT)

In [ ]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)

In [ ]:
def get_session_history(session_id: str, user_id: str) -> CosmosDBChatMessageHistory:
    cosmos = CosmosDBChatMessageHistory(
        cosmos_endpoint=os.environ['AZURE_COSMOSDB_ENDPOINT'],
        cosmos_database=os.environ['AZURE_COSMOSDB_NAME'],
        cosmos_container=os.environ['AZURE_COSMOSDB_CONTAINER_NAME'],
        connection_string=os.environ['AZURE_COMOSDB_CONNECTION_STRING'],
        session_id=session_id,
        user_id=user_id
        )

    # prepare the cosmosdb instance
    cosmos.prepare_cosmos()
    return cosmos


In [ ]:
brain_agent_executor = RunnableWithMessageHistory(
    agent_executor,
    get_session_history,
    input_messages_key="question",
    history_messages_key="history",
    history_factory_config=[
        ConfigurableFieldSpec(
            id="user_id",
            annotation=str,
            name="User ID",
            description="Unique identifier for the user.",
            default="",
            is_shared=True,
        ),
        ConfigurableFieldSpec(
            id="session_id",
            annotation=str,
            name="Session ID",
            description="Unique identifier for the conversation.",
            default="",
            is_shared=True,
        ),
    ],
)

In [ ]:
# This is where we configure the session id and user id
random_session_id = "session"+ str(random.randint(1, 1000))
ramdom_user_id = "user"+ str(random.randint(1, 1000))

config={"configurable": {"session_id": random_session_id, "user_id": ramdom_user_id}}
print(random_session_id, ramdom_user_id)

### Let's talk to our GPT Smart Search Engine chat bot now

In [ ]:
# This question should not use any tool, the brain agent should answer it without the use of any tool
printmd(brain_agent_executor.invoke({"question": "Hi, I'm Pablo Marin, how are you doing today?"}, config=config)["output"])

In [ ]:
printmd(brain_agent_executor.invoke({"question": "what is your name and what do you do?"}, config=config)["output"])

In [ ]:
printmd(brain_agent_executor.invoke({"question": "booksearch, Can I move indexes?"}, 
                                    config=config)["output"])

In [ ]:
printmd(brain_agent_executor.invoke({"question": "chatgpt, tell me the formula in physics for momentum"}, config=config)["output"])

In [ ]:
printmd(brain_agent_executor.invoke({"question": "docsearch, what is the responsibility of Manager of Human Resources?"}, config=config)["output"])

In [ ]:
printmd(brain_agent_executor.invoke({"question": "Thank you Jarvis!"}, config=config)["output"])

### Let's talk to our GPT Smart Search Engine chat bot with more questions and validate the responses from the chat bot

# Summary

방금 GPT 스마트 검색 엔진을 구축했습니다!
이 노트북에서 우리는 사용자의 질문에 답하기 위해 어떤 도구를 사용할지 결정하는 의사 결정 에이전트인 두뇌를 만들었습니다. 이것이 바로 스마트 채팅 봇을 만들기 위해 필요한 것이었습니다.

API에 연결하고, 파일 시스템을 다루고, 심지어 사람을 도구로 사용하는 등 다양한 작업을 수행할 수 있는 많은 도구가 있습니다. 자세한 내용은 [여기](https://python.langchain.com/docs/integrations/tools/)를 참조하세요.

# NEXT
이제 지금까지 빌드한 모든 기능과 프롬프트를 사용하여 웹 애플리케이션을 빌드할 차례입니다.
다음 노트북에서 빌드 방법을 안내해 드리겠습니다.


1) 봇 API 백엔드
2) 검색 및 웹챗 인터페이스가 있는 프런트엔드 UI